In [1]:
cd ..

/home/is/akiyoshi-n/my-project


In [2]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_dataset_2class_classification,load_dataset_4class_Multi_classification, split_multilabel_data, split_data_by_index, load_dataset_3class_Multi_classification, split_same_texts_data
from src.my_project.train_v2 import ActivityPrediction
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np

In [3]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

In [5]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 3
# 早期停止のための忍耐値
PATIENCE = 10
# 乱数シード
SEED = 2023
# クラス数
NUM_LABELS = 21
# 閾値
THRESH = 0.5

### Cross validation

In [6]:
single_raw_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [7]:
# データの読み込み
multi_data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [8]:
# testデータと訓練に使用するデータに分割
multi_data, multi_test_data = split_multilabel_data(data=multi_data, test_size=0.1, SEED=SEED)

In [9]:
dataset_labels_np = np.array(multi_data['labels'])
test_data_labels_np = np.array(multi_test_data['labels'])
print(dataset_labels_np.sum(axis=0))
print(test_data_labels_np.sum(axis=0))

[ 26  19  25  42   7  58  25  10  17   7  10 124  25  18  75  12  49  12
  10   5 550]
[ 3  2  3  5  1  6  3  1  2  1  1 14  3  2  8  1  5  1  1  1 61]


In [10]:
single_data = split_same_texts_data(multi_data, single_raw_data)
single_test_data = split_same_texts_data(multi_test_data, single_raw_data)

In [11]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_labels=21)

In [12]:
# Cross Validation
result = Classifier_model.cross_validation(single_data, multi_data, single_test_data, multi_test_data, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, NUM_FOLDS, output_dir, project_name='2step_cross_validation')

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/716 [00:00<?, ? examples/s]

Map:   0%|          | 0/359 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.666300,0.650205,0.626741,0.619053
2,0.606200,0.612671,0.668524,0.652101
3,0.495200,0.577585,0.715877,0.710484
4,0.382900,0.575890,0.729805,0.728380
5,0.195300,0.696774,0.682451,0.682429
6,0.083900,1.073210,0.710306,0.708404
7,0.023200,1.572449,0.704735,0.704715
8,0.023500,1.739670,0.701950,0.701199
9,0.055000,1.611284,0.701950,0.701802
10,0.011000,1.760708,0.710306,0.709112


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.644300,0.573674,0.000000,0.060277,"[0.0, 0.105, 0.2, 0.153, 0.0, 0.2, 0.0, 0.0, 0.065, 0.0, 0.037, 0.216, 0.0, 0.16, 0.0, 0.043, 0.0, 0.0, 0.0, 0.026]","[0.0, 0.286, 0.111, 0.643, 0.0, 0.35, 0.0, 0.0, 0.833, 0.0, 0.75, 0.286, 0.0, 0.333, 0.0, 0.75, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.065, 1.0, 0.087, 0.0, 0.14, 0.0, 0.0, 0.034, 0.0, 0.019, 0.174, 0.0, 0.105, 0.0, 0.022, 0.0, 0.0, 0.0, 0.013]"
2,0.482200,0.363222,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.294500,0.216236,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0.199900,0.164912,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,0.163400,0.141187,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.147900,0.128195,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,0.136000,0.120137,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,0.129100,0.114836,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,0.126700,0.111598,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
10,0.127600,0.109350,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

{'accuracy': 0.63, 'macro_f1': 0.23782399722249345, 'class_f1': [0.5, 0.0, 0.0, 0.909, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.752], 'class_recall': [0.5, 0.0, 0.0, 1.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.429, 0.0, 0.0, 0.0, 0.0, 0.0, 0.98], 'class_precision': [0.5, 0.0, 0.0, 0.833, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.61]}


In [14]:
# 重みありの場合の結果（cv=5）解答
average_accuracy = round(sum(d['accuracy'] for d in result)/len(result), 3)
average_macro_f1 = round(sum(d['macro_f1'] for d in result)/len(result), 3)
# クラスごとの平均値を計算
average_class_f1 = [round(sum(d['class_f1'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_f1']))]
average_class_recall = [round(sum(d['class_recall'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_recall']))]
average_class_precision = [round(sum(d['class_precision'][i] for d in result) / len(result), 3) for i in range(len(result[0]['class_precision']))]
print("Average accuracy:", average_accuracy)
print("Average Macro f1:", average_macro_f1)
print("Average Class f1:", average_class_f1)
print("Average Class recall:", average_class_recall)
print("Average Class precision:", average_class_precision)

Average accuracy: 0.651
Average Macro f1: 0.248
Average Class f1: [0.167, 0.0, 0.0, 0.856, 0.0, 0.444, 0.0, 0.0, 0.0, 0.0, 1.0, 0.154, 0.0, 0.667, 0.591, 0.0, 0.221, 0.0, 0.333, 0.0, 0.771]
Average Class recall: [0.167, 0.0, 0.0, 1.0, 0.0, 0.333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.111, 0.0, 0.667, 0.553, 0.0, 0.267, 0.0, 0.333, 0.0, 0.928]
Average Class precision: [0.167, 0.0, 0.0, 0.753, 0.0, 0.667, 0.0, 0.0, 0.0, 0.0, 1.0, 0.25, 0.0, 0.667, 0.672, 0.0, 0.192, 0.0, 0.333, 0.0, 0.663]


### ursoデータで予測

In [6]:
import re
import pandas as pd

# 正規表現パターンと対応する置換表現を辞書で定義
replacements = {
    r"@(\S*)": "[USR]", # ユーザ名を[USR]に置換
    r"http(\S*)": "[URL]", # URLを[URL]に置換
}

# 文中のマッチした箇所を対応する置換表現に置換する関数
def apply_regex(text):
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text

# テキストファイルを読み込み、1つ目と3つ目のフィールドに前処理を適用し、それらを出力する関数
def preprocess_file(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        processed_data = []
        for line in file:
            fields = line.strip().split('\t')
            # 2つ目のフィールド（前処理なし）と3つ目のフィールド（前処理あり）4つ目のフィールド（前処理なし）を取得
            first_field = fields[1] # ユーザ名
            third_field = apply_regex(fields[2]) # ツイート本文
            forth_field = fields[3] # ツイート日時
            processed_data.append([first_field, third_field, forth_field])
    return processed_data

In [7]:
# 関数を使用してファイルを前処理し、結果をDataFrameに格納
input_file = f'{DATASET_PATH}/urso_users.txt' # 入力ファイル名
processed_data = preprocess_file(input_file)
column_names = ['Name','text','time']
# DataFrameを作成
df = pd.DataFrame(processed_data, columns=column_names)

In [8]:
import pandas as pd
from transformers import AutoTokenizer
from src.my_project.dataset import preprocess_for_Trainer
import numpy as np
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
# tokenizerの定義
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# テキストデータを取得
texts = df['text'].values.tolist()
# テキストを含む辞書型のデータセットとして返す
test_dataset = {
    'texts': texts
}

In [9]:
single_raw_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [10]:
# データの読み込み
multi_data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [11]:
# testデータと訓練に使用するデータに分割
multi_data, multi_eval_data = split_multilabel_data(data=multi_data, test_size=0.2, SEED=SEED)

In [12]:
dataset_labels_np = np.array(multi_data['labels'])
test_data_labels_np = np.array(multi_eval_data['labels'])
print(dataset_labels_np.sum(axis=0))
print(test_data_labels_np.sum(axis=0))

[ 23  17  22  38   6  51  22   9  15   6   9 110  22  16  66  10  43  10
   9   5 489]
[  6   4   6   9   2  13   6   2   4   2   2  28   6   4  17   3  11   3
   2   1 122]


In [13]:
single_data = split_same_texts_data(multi_data, single_raw_data)
single_eval_data = split_same_texts_data(multi_eval_data, single_raw_data)

In [14]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_labels=21)

In [15]:
prediction = Classifier_model.train_and_predict(single_data, single_eval_data, multi_data, multi_eval_data, test_dataset, test_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='2step_train_and_predict')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/954 [00:00<?, ? examples/s]

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.703500,0.663919,0.626016,0.614788
2,0.617400,0.601487,0.695122,0.694996
3,0.525100,0.539312,0.731707,0.731548
4,0.392700,0.601269,0.715447,0.715447
5,0.228000,0.829631,0.699187,0.695789
6,0.120400,0.862107,0.723577,0.723559
7,0.033300,1.552369,0.699187,0.688288
8,0.015600,1.562827,0.739837,0.739407
9,0.005700,1.772706,0.707317,0.705742
10,0.028900,1.738644,0.719512,0.719508


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.631200,0.547102,0.000000,0.048150,"[0.0, 0.0, 0.0, 0.147, 0.0, 0.128, 0.0, 0.0, 0.022, 0.0, 0.017, 0.333, 0.167, 0.0, 0.111, 0.038, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.556, 0.0, 0.231, 0.0, 0.0, 0.25, 0.0, 0.5, 0.429, 0.167, 0.0, 0.059, 0.667, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.085, 0.0, 0.088, 0.0, 0.0, 0.011, 0.0, 0.009, 0.273, 0.167, 0.0, 1.0, 0.02, 0.0, 0.0, 0.0, 0.0]"
2,0.429700,0.297585,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.242800,0.184984,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0.172600,0.146454,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,0.145100,0.128977,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.131100,0.120426,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,0.127000,0.115593,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,0.123900,0.112781,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,0.121100,0.111256,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
10,0.117500,0.109267,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


Map:   0%|          | 0/510809 [00:00<?, ? examples/s]

Map:   0%|          | 0/238872 [00:00<?, ? examples/s]

In [16]:
# predictionをpickleで保存
import pickle
with open(f'{DATASET_PATH}/urso_2step_21class_pred.pkl', 'wb') as f:
    pickle.dump(prediction, f)

In [19]:
# 列の合計が0の個数
print(np.array(prediction).sum(axis=1).tolist().count(0))
print(len(prediction))

158149
510809


In [20]:
new_df = pd.DataFrame(prediction)
new_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510804,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
510805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
510806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
510807,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
action_df = pd.concat([df, new_df], axis=1)

In [22]:
action_df = action_df[['Name',0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]

In [24]:
# 0~20の列の合計が0の行を削除
action_df = action_df.drop(action_df.index[np.array(prediction).sum(axis=1)==0])

In [26]:
action_df.shape

(352660, 22)

In [27]:
# ユーザ名でグループ化し，平均を計算
action_df.groupby('Name').mean().mean(axis=0)

0     0.019719
1     0.000347
2     0.006475
3     0.026352
4     0.000000
5     0.030263
6     0.016732
7     0.000026
8     0.001288
9     0.000000
10    0.000000
11    0.063865
12    0.009202
13    0.000015
14    0.022291
15    0.000000
16    0.054044
17    0.000208
18    0.000000
19    0.000000
20    0.749174
dtype: float64

In [31]:
# ユーザ名でグループ化し，分散を計算
action_df.groupby('Name').mean().var(axis=0)

0     4.018090e-04
1     1.203019e-06
2     8.116329e-05
3     3.203076e-04
4     0.000000e+00
5     6.893633e-04
6     4.291445e-04
7     3.417878e-08
8     5.492376e-06
9     0.000000e+00
10    0.000000e+00
11    3.131198e-03
12    7.009645e-05
13    1.118548e-08
14    8.075417e-04
15    0.000000e+00
16    4.061331e-03
17    1.331098e-06
18    0.000000e+00
19    0.000000e+00
20    1.115395e-02
dtype: float64

In [8]:
# データの読み込み
single_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final.xlsx")

In [9]:
# 訓練データと評価データを辞書型で抽出
single_train_dataset = {
    'texts': [single_data['texts'][i] for i in range(900)],
    'labels': [single_data['labels'][i] for i in range(900)]
}
single_eval_dataset = {
    'texts': [single_data['texts'][i] for i in range(900, 1100)],
    'labels': [single_data['labels'][i] for i in range(900, 1100)]
}

In [10]:
# データの読み込み
multi_data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final.xlsx")

In [13]:
# 訓練データと評価データを辞書型で抽出
multi_train_dataset = {
    'texts': [multi_data['texts'][i] for i in range(900)],
    'labels': [multi_data['labels'][i] for i in range(900)]
}
multi_eval_dataset = {
    'texts': [multi_data['texts'][i] for i in range(900, 1100)],
    'labels': [multi_data['labels'][i] for i in range(900, 1100)]
}

### モデル

In [14]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_single_labels=2, num_multi_labels=19)

In [15]:
prediction = Classifier_model.train_and_predict(single_train_dataset, single_eval_dataset, multi_train_dataset, multi_eval_dataset, single_eval_dataset, multi_eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='2step_21class')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.733600,0.697466,0.485000,0.466321
2,0.670600,0.659246,0.610000,0.530120
3,0.598000,0.613886,0.660000,0.552632
4,0.517100,0.649763,0.695000,0.504065
5,0.392200,0.605030,0.685000,0.608696
6,0.265300,0.833193,0.690000,0.456140
7,0.157400,0.840043,0.715000,0.564885


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/428 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.741100,0.705353,0.000000,0.109293
2,0.627100,0.536552,0.013889,0.040019
3,0.476600,0.398403,0.013889,0.008772
4,0.363200,0.310921,0.000000,0.000000
5,0.299500,0.264546,0.000000,0.000000
6,0.262600,0.236902,0.000000,0.000000
7,0.240800,0.220976,0.000000,0.000000
8,0.227600,0.210076,0.000000,0.000000
9,0.218500,0.202154,0.000000,0.000000
10,0.211800,0.196317,0.000000,0.000000


KeyboardInterrupt: 